In [1]:
from FedAvg_models import *
from FedProx_models import *
from utils import *

In [2]:
fraction_for_test = 0.15
num_class = 4
ALL_MOTION = [i for i in range(1, num_class+1)]
N_MOTION = len(ALL_MOTION) # Number of output classes
T_MAX = 38 # Number of timestamps
n_gru_hidden_units = 128
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

destination_folder = './results'
try:
    os.makedirs(destination_folder)
except:
    pass

# Model

In [3]:
class CNNModule(nn.Module):
    def __init__(self):
        super(CNNModule, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=2, padding='same'),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(8 * 10 * 10, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, 32),
            nn.ReLU()
        )

    def forward(self, x):
        return self.cnn(x)

class ConvGRUModel(nn.Module):
    def __init__(self, hidden_size, num_classes, num_timestamps):
        super(ConvGRUModel, self).__init__()

        # CNN module for each input timestamp
        self.cnn_modules = nn.ModuleList([
            CNNModule() for _ in range(num_timestamps)
        ])

        # GRU layers
        self.gru = nn.GRU(32, hidden_size, num_layers=num_timestamps, batch_first=True, dropout=0.25)

        # Fully connected layer at the output of last GRU
        self.fc_out = nn.Linear(hidden_size, num_classes)

        # Relu activation for fully connected
        self.relu = nn.ReLU()
        # Softmax activation for classification
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Apply CNN module sequentially for each timestamp
        x = x.unsqueeze(2)
        x = np.swapaxes(x, 0, 1)
        x = [module(xi) for module, xi in zip(self.cnn_modules, x)]
        x = torch.stack(x, dim=1)  # Stack along the time dimension

        # GRU layer
        x, _ = self.gru(x)

        # Apply ReLU activation after the GRU layer
        x = self.relu(x)

        # Fully connected layer at the output of last GRU
        x = self.fc_out(x[:, -1, :])

        # Softmax for classification
        x = self.softmax(x)

        return x

## Load dataset

In [4]:
# Load datasets
num_clients = 5
batch_size = 128
client_datasets = {}
client_loaders = {}

for i in range(1, num_clients + 1):
    # Load client data
    client_data = torch.load(f'./data/data{i}.pt')
    data = torch.from_numpy(client_data['data']).float()
    label = torch.from_numpy(client_data['label']).long()

    # Split data into training and testing sets
    data_train, data_test, label_train, label_test = train_test_split(
        data, label, test_size=fraction_for_test, random_state=42
    )

    train_dataset = TensorDataset(data_train, label_train)
    test_dataset = TensorDataset(data_test, label_test)
    client_datasets[f'client{i}'] = {'train': train_dataset, 'test':test_dataset}

    # Set up data loaders for each client's
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    client_loaders[f'client{i}'] = {'train': train_loader, 'test':test_loader}

# Evasion Attack

In [5]:
def evasion_attack(data, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_data = data + epsilon * sign_data_grad
    perturbed_data = torch.clamp(perturbed_data, 0, 1)
    return perturbed_data

def evasion_attack_test(model, device, client_loaders, epsilon= 0.2):
    test_size = []
    clients_accuracye = []

    for client in range(1, len(client_loaders)+1):
        test_loader = client_loaders[f'client{client}']['test']
        test_size.append(len(test_loader.dataset))

        correct = 0
        total = 0
        for data, labels in test_loader:
            model.to(device)
            model.train()

            data, labels = data.to(device), labels.to(device)

            data.requires_grad = True
            outputs = model(data)
            
            loss = nn.CrossEntropyLoss()(outputs, labels)
            model.zero_grad()
            loss.backward()

            data_grad = data.grad.data
            perturbed_images = evasion_attack(data, epsilon, data_grad)

            outputs = model(perturbed_images)
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100 * correct / total
        clients_accuracye.append(accuracy)
    
    accuracy = np.average(clients_accuracye, weights=test_size)
    print(f'Weighted mean Accuracy on clients test data after evasion attack: {accuracy:.2f}%')


### FedAvg

In [6]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedAvg/FedAvg_model.pth')
model.load_state_dict(loaded_model)

evasion_attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedAvg+SAM

In [7]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedAvg/FedAvg_SAM_model.pth')
model.load_state_dict(loaded_model)

evasion_attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedAvg+FGSM

In [8]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedAvg/FedAvg_FGSM_model.pth')
model.load_state_dict(loaded_model)

evasion_attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 30.93%


### FedProx

In [9]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedProx/FedProx_model.pth')
model.load_state_dict(loaded_model)

evasion_attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedProx+SAM

In [10]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedProx/FedProx_SAM_model.pth')
model.load_state_dict(loaded_model)

evasion_attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedProx+FGSM

In [11]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedProx/FedProx_FGSM_model.pth')
model.load_state_dict(loaded_model)

evasion_attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedADMM

In [12]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedADMM/FedADMM_model.pth')
model.load_state_dict(loaded_model)

evasion_attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 27.76%


# Adversarial Attack

In [13]:
def adversarial_attack(data, mean=0, std=0.4):

    noise = torch.randn(data.shape) * std + mean
    noisy_data = data + 0.1 * noise
    # Ensure the pixel values are still in the valid range [0, 1]
    noisy_data = torch.clamp(noisy_data, 0, 1)
    return noisy_data


def attack_test(model, device, client_loaders, mean=0, std=0.75):
    test_size = []
    clients_accuracye = []

    for client in range(1, len(client_loaders)+1):
        test_loader = client_loaders[f'client{client}']['test']
        test_size.append(len(test_loader.dataset))

        correct = 0
        total = 0
        for data, labels in test_loader:
            model.to(device)
            model.eval()

            data, labels = data, labels.to(device)
                        
            perturbed_images = adversarial_attack(data, mean, std).to(device)

            outputs = model(perturbed_images)
            
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100 * correct / total
        clients_accuracye.append(accuracy)
    
    accuracy = np.average(clients_accuracye, weights=test_size)
    print(f'Weighted mean Accuracy on clients test data after evasion attack: {accuracy:.2f}%')


### FedAvg

In [14]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedAvg/FedAvg_model.pth')
model.load_state_dict(loaded_model)

attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedAvg+SAM

In [15]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedAvg/FedAvg_SAM_model.pth')
model.load_state_dict(loaded_model)

attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedAvg+FGSM

In [16]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedAvg/FedAvg_FGSM_model.pth')
model.load_state_dict(loaded_model)

attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedProx

In [17]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedProx/FedProx_model.pth')
model.load_state_dict(loaded_model)

attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedProx+SAM

In [18]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedProx/FedProx_SAM_model.pth')
model.load_state_dict(loaded_model)

attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedProx+FGSM

In [19]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedProx/FedProx_FGSM_model.pth')
model.load_state_dict(loaded_model)

attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 31.02%


### FedADMM

In [20]:
model = ConvGRUModel(n_gru_hidden_units, N_MOTION, T_MAX)
loaded_model = torch.load('./results/FedADMM/FedADMM_model.pth')
model.load_state_dict(loaded_model)

attack_test(model, device, client_loaders)

Weighted mean Accuracy on clients test data after evasion attack: 27.98%
